### Get details for multiple single shows
In this example we get the details of multiple single shows simultaneously.

First, we need to install the latest [Trakt.NET](https://www.nuget.org/packages/Trakt.NET) NuGet package.

In [ ]:
#r "nuget:Trakt.NET"

Now, we can create a new Trakt.NET client.

Since we do not need authentication or authorization for this example, only the Client-ID is required.

In [ ]:
using TraktNet;

string clientID = await GetInputAsync("Please enter your Trakt Client-ID:");

var client = new TraktClient(clientID);

##### Exception Handling

This example uses exceptions, which is the default for using the Trakt.NET library.

If we do not want to use exceptions, we can disable this behaviour with the following setting:
```csharp
client.Configuration.ThrowResponseExceptions = false;
```

We then need to check whether a response is valid like this:
```csharp
TraktResponse<ITraktShow> showResponse = await client.Shows.GetShowAsync(...);

if (showResponse) // Or showResponse.IsSuccess
{
    // Do something with the response.
}
else
{
    // Get a possible thrown exception
    var exception = showResponse.Exception;
}
```

Set the Trakt-IDs or -Slugs for the shows.

In [ ]:
string showIdOrSlug1 = await GetInputAsync("Enter the Trakt-Id or -Slug of Show 1:");
string showIdOrSlug2 = await GetInputAsync("Enter the Trakt-Id or -Slug of Show 2:");
string showIdOrSlug3 = await GetInputAsync("Enter the Trakt-Id or -Slug of Show 3:");

// Default fallback slugs.
showIdOrSlug1 = string.IsNullOrEmpty(showIdOrSlug1) ? "game-of-thrones" : showIdOrSlug1;
showIdOrSlug2 = string.IsNullOrEmpty(showIdOrSlug2) ? "mr-robot" : showIdOrSlug2;
showIdOrSlug3 = string.IsNullOrEmpty(showIdOrSlug3) ? "breaking-bad" : showIdOrSlug3;

The following lines show how we can get the details of multiple shows in one library call.
1. In line 11 we create a `TraktMultipleObjectsQueryParams` instance, which holds the ids of the shows.
2. We use the `TraktMultipleObjectsQueryParams` for the request (see line 20).

In [ ]:
using TraktNet.Exceptions;
using TraktNet.Modules;
using TraktNet.Objects.Get.Shows;
using TraktNet.Parameters;
using TraktNet.Responses;

try
{
    var extendedInfo = new TraktExtendedInfo { Full = true };

    var parameters = new TraktMultipleObjectsQueryParams
    {
        showIdOrSlug1, // For this show we want only the minimal information.

        // The following shows will have full information.
        { showIdOrSlug2, extendedInfo },
        { showIdOrSlug3, extendedInfo }
    };

    IEnumerable<TraktResponse<ITraktShow>> mutlipleShowsResponse = await client.Shows.GetMultipleShowsAsync(parameters);

    foreach (TraktResponse<ITraktShow> showResponse in mutlipleShowsResponse)
    {
        ITraktShow show = showResponse.Value;

        Console.WriteLine("-------------------------------------------------------------------------");
        
        Console.WriteLine($"Title: {show.Title}");
        Console.WriteLine($"Year: {show.Year ?? 0}");

        ITraktShowIds ids = show.Ids;

        if (ids != null)
        {
            Console.WriteLine($"Trakt-Id: {ids.Trakt}");
            Console.WriteLine($"Slug: {ids.Slug}");
            Console.WriteLine($"ImDB-Id: {ids.Imdb}");
            Console.WriteLine($"TmDB-Id: {ids.Tmdb ?? 0}");
            Console.WriteLine($"TVDB-Id: {ids.Tvdb ?? 0}");
            Console.WriteLine($"TVRage-Id: {ids.TvRage ?? 0}");
        }

        Console.WriteLine($"Overview: {show.Overview}");

        if (show.FirstAired.HasValue)
            Console.WriteLine($"First Aired (UTC): {show.FirstAired.Value}");

        ITraktShowAirs airs = show.Airs;

        if (airs != null)
        {
            Console.WriteLine($"Airs on: {airs.Day}");
            Console.WriteLine($"Airs at: {airs.Time}");
            Console.WriteLine($"Airs in: {airs.TimeZoneId}");
        }

        Console.WriteLine($"Runtime: {show.Runtime ?? 0} Minutes");

        if (show.Genres != null)
            Console.WriteLine($"Genres: {string.Join(", ", show.Genres)}");

        Console.WriteLine($"Certification: {show.Certification}");
        Console.WriteLine($"Network: {show.Network}");
        Console.WriteLine($"Aired Episodes: {show.AiredEpisodes ?? 0}");

        if (show.Status != null)
            Console.WriteLine($"Status: {show.Status.DisplayName}");

        Console.WriteLine($"Rating: {show.Rating ?? 0.0f}");
        Console.WriteLine($"Votes: {show.Votes ?? 0}");
        Console.WriteLine($"Country Code: {show.CountryCode}");
        Console.WriteLine($"Language Code: {show.LanguageCode}");

        if (show.UpdatedAt.HasValue)
            Console.WriteLine($"Updated At (UTC): {show.UpdatedAt.Value}");

        if (show.AvailableTranslationLanguageCodes != null)
            Console.WriteLine($"Available Translation Languages: {string.Join(", ", show.AvailableTranslationLanguageCodes)}");

        Console.WriteLine($"Trailer: {show.Trailer}");
        Console.WriteLine($"Homepage: {show.Homepage}");
    }
}
catch (TraktException ex)
{
    Console.WriteLine("-------------- Trakt Exception --------------");
    Console.WriteLine($"Exception message: {ex.Message}");
    Console.WriteLine($"Status code: {ex.StatusCode}");
    Console.WriteLine($"Request URL: {ex.RequestUrl}");
    Console.WriteLine($"Request message: {ex.RequestBody}");
    Console.WriteLine($"Request response: {ex.Response}");
    Console.WriteLine($"Server Reason Phrase: {ex.ServerReasonPhrase}");
    Console.WriteLine("---------------------------------------------");
}